In [2]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4
%pip install --upgrade --quiet  langchain sentence_transformers langchain_groq -U langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain_community.document_loaders import JSONLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
import json
from pathlib import Path
from pprint import pprint, PrettyPrinter
from typing import Dict
from pydantic import BaseModel

In [4]:
with open('/content/data.json') as f:
  data = json.load(f)

In [5]:
class Document(BaseModel):
    page_content:str
    metadata:Dict

In [6]:
docs=[]
for index,doc in enumerate(data):
    docs.append(Document(page_content=str(doc),metadata={'source': '/content/data.json', 'seq_num': index+1}))

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# Equivalent to SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# save to disk
db2 = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")

In [9]:
retriever = db2.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [10]:
retrieved_docs = retriever.invoke("heart specialist")

In [ ]:
retrieved_docs

[Document(page_content="{'doctor-profile-link': 'https://oladoc.com/pakistan/multan/dr/internal-medicine-specialist/burhan-pasha/11449', 'doctor-name': 'Dr. Muhammad Burhan Pasha', 'satisfaction': '', 'total-reviews': '', 'services': ['Cardiac Catheterization (کارڈیک کیتھرائزیشن)', 'Cardiac Patient Management (کارڈیک مریضوں کا انتظام)', 'Cardiology Medicine (General) (امراض قلب (عام))', 'COVID-19 / Corona Virus Treatment', 'Diabetes Management (ذیابیطس کا انتظام)', 'Heart Attack Management (ہارٹ اٹیک مینجمنٹ)', 'Hypertension Specialist'], 'conditios_treated': '', 'doctor_education': 'MBBS\\nFCPS (Medicine)\\nMCPS (Cardio)', 'doc_clinic_staff_behavior': '', 'about_doc_info': 'About Dr. Muhammad Burhan Pasha\\nDr. Muhammad Burhan Pasha is an expert Internal Medicine Specialist with 14 years of experience currently practicing at Haleema Hospital Complex, Multan. You can call our helpline 0618048444 for more information.\\nExperience\\nDr. Muhammad Burhan Pasha has over 14 years of experie

In [11]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "".join(doc.page_content for doc in docs)

example_messages = prompt.invoke(
    {"context": f"...", "question": "COVID-19"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: COVID-19 \nContext: ... \nAnswer:")]

In [ ]:
from dotenv import find_dotenv, load_dotenv
import os

dotenv_path = '.env'
load_dotenv(dotenv_path)
GROQ_API = os.getenv('GROQ_API')

In [12]:
llm = ChatGroq(temperature=0.5,
                model_name="Llama3-70b-8192",
                api_key=GROQ_API,
                max_tokens=100,
                model_kwargs={
                    "top_p": 1,
                    "frequency_penalty": 0.0,
                    "presence_penalty": 0.0
                }
                )

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.


{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("Doctor?")

In [22]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)



qa_system_prompt = """You are a helpful AI Assistant Your name is HealthConnect AI and you goal is to Assist the Patients by proding and best doctors recommendations. \
welcome to the patients with this message "Hello! Welcome to HealthConnect AI, your personalized health assistant. I'm here to help you find the best doctor based on your needs and preferences."
## Instructions:
1:Location Preference:
In which city or region of Pakistan are you looking to find a doctor?
--This will help to find a doctor in a specfic location.

2:Type of Disease or Health Concern:
What type of disease or health concern are you seeking treatment for? (e.g., diabetes, cardiology, general check-up, etc.)
--This will help to know about the patient disease or health concern to reccomend a suitable doctor.


3:Budget for Doctor's Fee:
"What is your budget for the doctor's fee? Please specify an approximate range (e.g., PKR 500 - PKR 5000)."

4:Doctor's Gender Preference:
"Do you have a preference for the doctor's gender? (Male, Female, No preference)"

Do not ask the above question at once ask one by one in a professional tone.

Use the following pieces of retrieved context to answer the question. accroding to the gathered information. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [23]:
from langchain_core.messages import HumanMessage

chat_history = []

In [ ]:
while(True):
    print("*"*100)
    question=input("Human:")
    ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
    chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])
    print(f"HealthConnect AI: {ai_msg_1['answer']}")


In [20]:
second_question = "Given me information about doctor who has lowest treament fee?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
print(ai_msg_2["answer"])

According to the provided information, Dr. Mehran Khan has the lowest treatment fee, which is Rs. 500.


In [21]:
chat_history.extend([HumanMessage(content=question), ai_msg_2["answer"]])
third_question = "Give me url link to Dr.Mehran Khan?"
ai_msg_3 = rag_chain.invoke({"input": third_question, "chat_history": chat_history})
print(ai_msg_3["answer"])

The URL link to Dr. Mehran Khan's profile is https://oladoc.com/pakistan/mardan/dr/internal-medicine-specialist/mehran-khan/378191.
